# Worksheet 08

Name:  Jiawei Sun

UID: U81507478

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [72]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.065215084777658, 4.938873256251253, 4.591250820623465, 4.855201149652522, 6.688249872356684, 4.9717923810043905, 3.2733204449546567, 5.126483315605508, 3.31614537003014, 6.722984119137449]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [73]:
c2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(c2)

[8.278718939238741, 6.766304435540415, 9.806787310988987, 9.089345383023428, 9.421823626972738, 8.064721352700822, 7.38388685353142, 6.019217797233448, 7.187936557459981, 7.794441699191401]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [74]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[7.794441699191401, 6.722984119137449, 7.187936557459981, 6.019217797233448, 7.38388685353142, 3.31614537003014, 8.064721352700822, 5.126483315605508, 3.2733204449546567, 9.421823626972738]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- Mean, variance, and probability for every species.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [75]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[3.31614537003014, 5.126483315605508, 3.2733204449546567]
[7.794441699191401, 6.722984119137449, 7.187936557459981, 6.019217797233448, 7.38388685353142, 8.064721352700822, 9.421823626972738]
P(C_1) = 0.3,  P(C_2) = 0.7
mean_1 = 3.905316376863435,  mean_2 = 7.5135731437467514
var_1 = 0.7459300085062969,  var_2 = 1.0007225754409936


/Users/apple/Library/Python/3.11/lib/python/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [76]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_0 | X_i) = P(X_i | C_0)P(C_0) / P(X_i)
    prob_c0_x_i =  (pdf_i[0] * prob_c[0]) / prob_x
    prob_c0_x.append( prob_c0_x_i )
    
    # P(C_1 | X_i) = P(X_i | C_1)P(C_1) / P(X_i)
    prob_c1_x_i = (pdf_i[1] * prob_c[1]) / prob_x
    prob_c1_x.append( prob_c1_x_i )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  7.794441699191401
probability of observing that point if it came from cluster 0 =  6.688952255557601e-07
probability of observing that point if it came from cluster 1 =  0.3832577727677807
point =  6.722984119137449
probability of observing that point if it came from cluster 0 =  0.00042639324852414116
probability of observing that point if it came from cluster 1 =  0.29178874703224283
point =  7.187936557459981
probability of observing that point if it came from cluster 0 =  3.3334067891754796e-05
probability of observing that point if it came from cluster 1 =  0.37809722685539
point =  6.019217797233448
probability of observing that point if it came from cluster 0 =  0.009644677915340875
probability of observing that point if it came from cluster 1 =  0.13073303131753197
point =  7.38388685353142
probability of observing that point if it came from cluster 0 =  1.0135486499916882e-05
probability of observing that point if it came from cluster 1 =  0.3953206879862385
point =  

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [77]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [sum([x[0] * ((x[1] - mean[0])**2) for x in zip(prob_c0_x, data)]) / sum(prob_c0_x) , sum([x[0] * ((x[1] - mean[1])**2) for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.2752061902771862,  P(C_2) = 0.7247938097228137
mean_1 = 3.8060905225973207,  mean_2 = 7.427817910264067
var_1 = 0.7040089371899941,  var_2 = 1.1703127645157207


g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [78]:
prob_c0_x = []  # P(S_0 | X_i)
prob_c1_x = []  # P(S_1 | X_i)

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))

    # P(S_0 | X_i) = P(X_i | S_0)P(S_0) / P(X_i)
    prob_c0_x_i = (pdf_i[0] * prob_c[0]) / (pdf_i[0] * prob_c[0] + pdf_i[1] * prob_c[1])
    prob_c0_x.append(prob_c0_x_i)

    # P(S_1 | X_i) = P(X_i | S_1)P(S_1) / P(X_i)
    prob_c1_x_i = (pdf_i[1] * prob_c[1]) / (pdf_i[0] * prob_c[0] + pdf_i[1] * prob_c[1])
    prob_c1_x.append(prob_c1_x_i)

# Print updated posterior probabilities
probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_0 = " + str(p[1]))
    print("Probability of coming from S_1 = " + str(p[2]))
    print()

7.794441699191401
Probability of coming from S_0 = 7.116207070329158e-08
Probability of coming from S_1 = 0.9999999288379293

6.722984119137449
Probability of coming from S_0 = 0.00014163963303285098
Probability of coming from S_1 = 0.9998583603669672

7.187936557459981
Probability of coming from S_0 = 6.287882762629966e-06
Probability of coming from S_1 = 0.9999937121172374

6.019217797233448
Probability of coming from S_0 = 0.00922176120872344
Probability of coming from S_1 = 0.9907782387912766

7.38388685353142
Probability of coming from S_0 = 1.5566791445199546e-06
Probability of coming from S_1 = 0.9999984433208555

3.31614537003014
Probability of coming from S_0 = 0.9958038454962705
Probability of coming from S_1 = 0.004196154503729456

8.064721352700822
Probability of coming from S_0 = 8.29228652485188e-09
Probability of coming from S_1 = 0.9999999917077135

5.126483315605508
Probability of coming from S_0 = 0.4290990458277937
Probability of coming from S_1 = 0.5709009541722064


h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [79]:
hard_assignments = []  # List to store hard cluster assignments

for p_s0, p_s1 in zip(prob_c0_x, prob_c1_x):
    if p_s0 > p_s1:
        hard_assignments.append(0)  # Assign to cluster 0
    else:
        hard_assignments.append(1)  # Assign to cluster 1

# Print the hard assignments
for i, assignment in enumerate(hard_assignments):
    print(f"Data point {i} belongs to Cluster {assignment}")

Data point 0 belongs to Cluster 1
Data point 1 belongs to Cluster 1
Data point 2 belongs to Cluster 1
Data point 3 belongs to Cluster 1
Data point 4 belongs to Cluster 1
Data point 5 belongs to Cluster 0
Data point 6 belongs to Cluster 1
Data point 7 belongs to Cluster 1
Data point 8 belongs to Cluster 0
Data point 9 belongs to Cluster 1
